<a href="https://colab.research.google.com/github/Duaa-Raed/bookz/blob/main/bookz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========== IMPORTS (Clean + Organized) ==========

# Basic
import os
import pandas as pd
import numpy as np
import torch

# Colab utilities
from google.colab import files

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Embeddings & Models
from sentence_transformers import SentenceTransformer

# Vector Search
import faiss

# LangChain Tools (use only if needed)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS as LC_FAISS
from langchain_core.documents import Document

# HuggingFace models
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Gemini
import google.generativeai as genai


In [ ]:

#  Load the first file
print(" Upload the first file (e.g., books.csv):")
uploaded1 = files.upload()
df = pd.read_csv(list(uploaded1.keys())[0])
print(" File loaded:", list(uploaded1.keys())[0])
print(df.columns)
print(df.head())


In [ ]:
# Display the first 5 rows
print(" Data Overview:")
print(df.head())

# Quick info about columns
print("\n Column Information:")
print(df.info())

# Numerical statistics
print("\n General Statistics:")
print(df.describe())

# Check for missing values
print("\n Missing Values:")
print(df.isnull().sum())

In [ ]:
#  Exploratory Data Analysis (EDA)

# Configure font settings for Arabic display (if needed)
plt.rcParams['font.family'] = 'Arial'

# 1️ Most Productive Authors
top_authors = df['Author'].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_authors.values, y=top_authors.index, palette="viridis")
plt.title(" Most Productive Authors")
plt.xlabel("Number of Books")
plt.ylabel("Author")
plt.show()

# 2️ Most Common Categories
top_categories = df['Category'].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_categories.values, y=top_categories.index, palette="magma")
plt.title(" Most Common Categories")
plt.xlabel("Number of Books")
plt.ylabel("Category")
plt.show()

# 3️ Price and Pages Analysis
print(" Price Statistics:")
print(df['Price'].describe())

print("\n Pages Statistics:")
print(df['Pages'].describe())

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(df['Price'], bins=40, kde=True, color='orange')
plt.title("Price Distribution")

plt.subplot(1,2,2)
sns.histplot(df['Pages'], bins=40, kde=True, color='blue')
plt.title("Pages Distribution")

plt.show()

# 4️ Relationship between Price and Pages
plt.figure(figsize=(7,6))
sns.scatterplot(data=df, x="Pages", y="Price", alpha=0.6)
plt.title(" Relationship between Price and Pages")
plt.xlabel("Number of Pages")
plt.ylabel("Price (Dinar)")
plt.show()

In [ ]:
# Delete rows where pages or prices are zero
df_clean = df[(df['Pages'] > 0) & (df['Price'] > 0)]

print("Number of books after initial cleaning:", len(df_clean))


df_clean = df_clean[(df_clean['Price'] < 500) & (df_clean['Pages'] < 2000)]

print("After removing outliers:", len(df_clean))

print("Prices after cleaning:")
print(df_clean['Price'].describe())

print("\nPages after cleaning:")
print(df_clean['Pages'].describe())

In [ ]:
# Clean texts from null values
df.fillna("", inplace=True)

# Create a unified text column
df["text"] = (
    " Title: " + df["Title"].astype(str) +
    " Author: " + df["Author"].astype(str) +
    " Category: " + df["Category"].astype(str) +
    " Description: " + df["Description"].astype(str)
)

print("Texts prepared:", len(df))
print(df["text"].head(3))

In [ ]:

#  Load the data file (select the file from your device)
print(" Upload the books file (e.g., jamalon dataset.csv):")
uploaded = files.upload()

# Read the file
df1 = pd.read_csv(list(uploaded.keys())[0])
print(" File loaded:", list(uploaded.keys())[0])
print(df1.columns)

#  Create a unified text column for title and description
if "Title" in df1.columns and "Description" in df1.columns:
    df1["text"] = df1["Title"].fillna('') + " - " + df1["Description"].fillna('')
    print(" The 'text' column was created successfully!")
else:
    print(" Make sure you have 'Title' and 'Description' columns in your file.")


In [ ]:

# Load the model
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Create the embeddings
embeddings = model.encode(df1["text"].tolist(), show_progress_bar=True)

# Convert to float32 for FAISS
embeddings = np.array(embeddings).astype("float32")

# Build the search index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"Knowledge base created successfully! Number of books: {len(embeddings)}")

In [ ]:
# Create the index
index = faiss.IndexFlatL2(embeddings.shape[1])
# Add the embeddings to the index
index.add(embeddings)
print(" Search database built:", index.ntotal)

In [ ]:
def ask(query, k=3):
    query_emb = model.encode([query]).astype("float32")
    distances, indices = index.search(query_emb, k)
    results = df.iloc[indices[0]]
    for i, row in results.iterrows():
        print(f" {row['Title']} — {row['Author']}")
        print(f" {row['Description'][:250]}...")
        print("—" * 60)


In [ ]:

# We use a lightweight model that supports Arabic and English
# Note: Ensure you have the necessary environment setup (GPU, bitsandbytes, accelerate) to run this model locally.
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

def rag_answer(query, k=3):
    # Search for the nearest books
    query_emb = model.encode([query]).astype("float32")
    distances, indices = index.search(query_emb, k)
    results = df1.iloc[indices[0]]

    # Aggregate texts from the books
    context = "\n\n".join(
        [f" {r['Title']} — {r['Author']}\n{r['Description']}" for _, r in results.iterrows()]
    )

    # Input prompt for the model
    prompt = f"""
    User Question: {query}

    Texts retrieved from the books (Context):
    {context}

     Answer concisely and naturally, relying ONLY on the texts provided above.
    """

    # Generate the answer
    answer = generator(prompt, max_new_tokens=200)[0]['generated_text']
    print(" Answer:\n", answer)

In [ ]:
def rag_answer(question, retriever, model, tokenizer):
    #  Step 1: Retrieve the chunks (sources) most relevant to the question
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([d.page_content for d in docs])

    #  Step 2: Prepare the full prompt for the LLM (Question + Context)
    prompt = f"Question: {question}\n\nAvailable Information:\n{context}\n\nDetailed Answer:"

    #  Step 3: Let the model generate the answer based on the context
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=300)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

In [ ]:

# Load the books file
print(" Upload the books file (e.g., jamalon dataset.csv):")
uploaded = files.upload()

# Read the file
df = pd.read_csv(list(uploaded.keys())[0])
print(" File loaded:", list(uploaded.keys())[0])
print(df.columns)

# Create a unified text column
if "Title" in df.columns and "Description" in df.columns:
    df["text"] = df["Title"].fillna('') + " - " + df["Description"].fillna('')
    print(" The 'text' column was created successfully!")
else:
    print(" Make sure the 'Title' and 'Description' columns exist in the file.")

In [ ]:

# Define the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert DataFrame rows into LangChain Document objects
documents = [Document(page_content=row["text"]) for _, row in df.iterrows()]

# Create the FAISS vector store from the documents and embeddings
vectorstore = FAISS.from_documents(documents, embeddings)

# Create a retriever object for similarity search (top 3 results)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print(" Vector search base built successfully!")

In [ ]:

#  Load the books file
print(" Upload the books file (e.g., jamalon dataset.csv):")
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])
print(" File loaded:", list(uploaded.keys())[0])

# Create a unified text column
df.fillna("", inplace=True)
df["text"] = (
    " Title: " + df["Title"].astype(str) +
    " Author: " + df["Author"].astype(str) +
    " Category: " + df["Category"].astype(str) +
    " Description: " + df["Description"].astype(str)
)

#  Create the vector documents (Embeddings)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

documents = [Document(page_content=row["text"]) for _, row in df.iterrows()]
vectorstore = FAISS.from_documents(documents, embeddings)

# Create the retriever (search tool)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print(" Vector search base built successfully!")

In [ ]:

#  Load the generation model
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


In [ ]:
def ask_books_bot(query):
    #  Retrieve results from the database
    results = retriever.invoke(query)

    # Filter results to include only texts containing the requested book name
    filtered = [doc for doc in results if any(word in doc.page_content for word in query.split())]

    if not filtered:
        print(" Sorry, no information is available for this book.")
        return

    # Merge the found texts
    context = "\n\n".join([doc.page_content for doc in filtered])

    #  Prepare the prompt
    prompt = f"""You are an intelligent assistant specialized in books and novels.
Rely only on the information below to answer briefly and clearly without reiterating it.

Knowledge:
{context}

User question: {query}
Answer:"""

    #  Generate the answer
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=180)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean the output
    if "Answer:" in answer:
        answer = answer.split("Answer:")[-1].strip()

    # Check quality
    if not answer or len(answer) < 15:
        print(" Sorry, no information is available for this book.")
    else:
        print("🤖", answer)

In [ ]:
# ==========================================
#  Final Code (Cleaned and Simplified Version)
# ==========================================

# 1️ Gemini API Setup
print(" Get a free API Key from: https://makersuite.google.com/app/apikey")
api_key = input("Enter API Key: ").strip()
genai.configure(api_key=api_key)

#  Latest and most compatible model
gemini_model = genai.GenerativeModel('gemini-2.5-flash')
print(" Gemini activated!")

# 2️ Load Data
print("\n Upload the books file:")
# Make sure to re-upload the file or that it exists in the runtime environment
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

df.fillna("", inplace=True)
df["text"] = (
    df["Title"].astype(str) + " | " +
    df["Author"].astype(str) + " | " +
    df["Description"].astype(str)
)

# 3️ Create Embeddings
print("🔍 Creating search base...")

# Clean GPU memory
if 'embeddings' in locals() or 'embeddings' in globals():
    del embeddings
if 'model' in locals() or 'model' in globals():
    del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print(" GPU memory cleaned.")

# Use the correct model all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Keep batch size small
BATCH_SIZE = 16
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True, batch_size=BATCH_SIZE)

embeddings_array = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings_array.shape[1])
index.add(embeddings_array)
print(f" Ready! ({len(df)} books)")

# 4️ Search function + Gemini (Final output modification)
def ask_gemini(query, k=3):
    """Smart search with answers from Gemini"""

    # Search in the database and aggregate context (unchanged)
    query_emb = model.encode([query]).astype('float32')
    distances, indices = index.search(query_emb, k)

    books_info = []
    for idx in indices[0]:
        book = df.iloc[idx]
        books_info.append(f"""
 {book['Title']}
 Author: {book['Author']}
 Description: {book['Description'][:300]}
""")
    context = "\n\n".join(books_info)

    # Prepare Prompt for Gemini (unchanged)
    prompt = f"""You are an intelligent library assistant. Use the following information to answer:
{context}
User question: {query}
Provide a helpful and concise answer (3-4 sentences only), and mention relevant book titles."""

    try:
        # Get answer from Gemini
        response = gemini_model.generate_content(prompt)

        #  Modified parts for clean printing:
        print("\n**Answer:**")
        print(response.text)

        #  Suggested Books
        print("\n--- Suggested Books ---")

        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"{i+1}. {book['Title']} - {book['Author']}")
        # --------------------------------------------------

    except Exception as e:
        print(f" Error with Gemini: {e}")
        print(" Displaying results directly (generation failed):")
        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"\n{i+1}.  {book['Title']}")
            print(f"    {book['Author']}")
            print(f"    {book['Description'][:200]}...")

# ==========================================
#  Experiment (removing initial examples)
# ==========================================
print("\n\n System ready!")

# Interactive mode
print("\n\n Ask me any question (or 'exit' to quit):")
while True:
    #  Remove '\n' from input line to avoid duplicate empty line
    q = input(" Your question: ").strip()
    if q.lower() in ['exit', 'quit']:
        break
    if q:
        ask_gemini(q)